In [5]:
import os
import pandas as pd
import numpy as np

# Caminhos das pastas de origem e destino
source_folder = "../datasets-by-source"
destination_folder = "../datasets-by-source-missing"

# Cria a pasta de destino, se não existir
os.makedirs(destination_folder, exist_ok=True)

# Lista de arquivos na pasta de origem
files = [f for f in os.listdir(source_folder) if f.endswith('.csv')]

# Função para substituir 15% dos dados por np.nan
def replace_with_nan(data, column_name, percentage):
    data = data.copy()
    n_missing = int(len(data) * percentage)
    missing_indices = np.random.choice(data.index, n_missing, replace=False)
    data.loc[missing_indices, column_name] = np.nan
    return data

# Processa cada arquivo na pasta de origem
for file in files:
    file_path = os.path.join(source_folder, file)
    df = pd.read_csv(file_path)

    # Remove as colunas especificadas, se existirem
    columns_to_remove = ["Link_bottleneck", "Unnamed: 0"]
    df = df.drop(columns=[col for col in columns_to_remove if col in df.columns], errors='ignore')

    # Substitui 15% dos dados nas colunas especificadas
    if 'Vazao' in df.columns:
        df = replace_with_nan(df, 'Vazao', 0.15)
    if 'Vazao_bbr' in df.columns:
        df = replace_with_nan(df, 'Vazao_bbr', 0.15)

    # Salva o dataset modificado na pasta de destino
    output_path = os.path.join(destination_folder, file)
    df.to_csv(output_path, index=False)

print("Processamento concluído. Os datasets foram salvos na pasta 'datasets-by-source-missing'.")


Processamento concluído. Os datasets foram salvos na pasta 'datasets-by-source-missing'.


In [6]:
import os
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression

# Caminhos das pastas de origem e destino
source_folder = "../datasets-by-source-missing"
imputed_folder = "../datasets-by-source-imputed"
os.makedirs(imputed_folder, exist_ok=True)
subfolders = ["knn", "regression", "time-weighted"]
for subfolder in subfolders:
    os.makedirs(os.path.join(imputed_folder, subfolder), exist_ok=True)

# Função para imputar dados com KNN
def impute_with_knn(data, columns, n_neighbors=5):
    imputer = KNNImputer(n_neighbors=n_neighbors)
    data[columns] = imputer.fit_transform(data[columns])
    return data

# Função para imputar dados com regressão
def impute_with_regression(data, columns):
    data = data.copy()
    for column in columns:
        if data[column].isna().sum() > 0:
            not_nan = data[data[column].notna()]
            nan = data[data[column].isna()]
            if not_nan.shape[0] > 0:
                features = not_nan.drop(columns, axis=1).select_dtypes(include=[np.number])
                target = not_nan[column]
                model = LinearRegression()
                model.fit(features, target)
                predicted = model.predict(nan.drop(columns, axis=1).select_dtypes(include=[np.number]))
                data.loc[data[column].isna(), column] = predicted
    return data

# Função para imputar dados com média ponderada pelo tempo
def impute_with_time_weighted(data, columns, time_column):
    data = data.copy()
    for column in columns:
        if data[column].isna().sum() > 0:
            for idx in data[data[column].isna()].index:
                row_time = data.loc[idx, time_column]
                weighted_sum = 0
                weight_sum = 0
                for _, row in data[data[column].notna()].iterrows():
                    time_diff = abs(row[time_column] - row_time)
                    weight = 1 / (time_diff + 1e-9)
                    weighted_sum += weight * row[column]
                    weight_sum += weight
                data.loc[idx, column] = weighted_sum / weight_sum
    return data

# Processa cada arquivo na pasta de origem
files = [f for f in os.listdir(source_folder) if f.endswith('.csv')]
columns_to_impute = ["Vazao", "Vazao_bbr"]
time_column = "Timestamp"

for file in files:
    file_path = os.path.join(source_folder, file)
    df = pd.read_csv(file_path)

    # Imputação com KNN
    df_knn = impute_with_knn(df.copy(), columns_to_impute)
    knn_path = os.path.join(imputed_folder, "knn", file)
    df_knn.to_csv(knn_path, index=False)

    # Imputação com regressão
    df_regression = impute_with_regression(df.copy(), columns_to_impute)
    regression_path = os.path.join(imputed_folder, "regression", file)
    df_regression.to_csv(regression_path, index=False)

    # Imputação com média ponderada pelo tempo
    if time_column in df.columns:
        df_time_weighted = impute_with_time_weighted(df.copy(), columns_to_impute, time_column)
        time_weighted_path = os.path.join(imputed_folder, "time-weighted", file)
        df_time_weighted.to_csv(time_weighted_path, index=False)

print("Processamento concluído. Os datasets imputados foram salvos nas subpastas 'knn', 'regression' e 'time-weighted'.")


Processamento concluído. Os datasets imputados foram salvos nas subpastas 'knn', 'regression' e 'time-weighted'.
